<a href="https://colab.research.google.com/github/LeeJaeEun0/230102_mini_project_st/blob/main/230102_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 신용카드 연체 예측 AI

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

파일 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/data/'
train = pd.read_csv(path + 'train.csv') # train 파일만 사용

데이터 확인

In [ ]:
train

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,F,N,N,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,-12079,-1984,1,0,0,0,Core staff,4.0,-2.0,1.0
26453,26453,F,N,Y,1,180000.0,Working,Higher education,Separated,House / apartment,-15291,-2475,1,0,0,0,NaN,2.0,-47.0,2.0
26454,26454,F,Y,N,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,-10082,-2015,1,0,0,0,Core staff,2.0,-25.0,2.0
26455,26455,M,N,Y,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,-10145,-107,1,0,0,0,Laborers,1.0,-59.0,2.0


## 1. 전처리하기

### 1. 칼럼 삭제하기 - index, FLAG_MOBIL, occyp_type

In [ ]:
train.drop(['index', 'FLAG_MOBIL','occyp_type','child_num'], axis=1, inplace=True)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gender         26457 non-null  object 
 1   car            26457 non-null  object 
 2   reality        26457 non-null  object 
 3   income_total   26457 non-null  float64
 4   income_type    26457 non-null  object 
 5   edu_type       26457 non-null  object 
 6   family_type    26457 non-null  object 
 7   house_type     26457 non-null  object 
 8   DAYS_BIRTH     26457 non-null  int64  
 9   DAYS_EMPLOYED  26457 non-null  int64  
 10  work_phone     26457 non-null  int64  
 11  phone          26457 non-null  int64  
 12  email          26457 non-null  int64  
 13  family_size    26457 non-null  float64
 14  begin_month    26457 non-null  float64
 15  credit         26457 non-null  float64
dtypes: float64(4), int64(5), object(7)
memory usage: 3.2+ MB


### 2. 라벨 인코더 사용하기

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
train['income_type'] = le.fit_transform(train['income_type'])
train['edu_type'] = le.fit_transform(train['edu_type'])
train['family_type'] = le.fit_transform(train['family_type'])
train['house_type'] = le.fit_transform(train['house_type'])

### 3. 원핫 인코딩

In [ ]:
train['gender'] = train['gender'].replace(['F','M'],[0,1])
train['car'] = train['car'].replace(['N','Y'],[0,1])
train['reality'] = train['reality'].replace(['N','Y'],[0,1])

### 4. DAYS_BIRTH, DAYS_EMPLOYED, begin_month - 양수처리

In [ ]:
# 0 이상의 값 0으로
train.loc[train['DAYS_EMPLOYED'] > 0, 'DAYS_EMPLOYED'] = 0
# 절댓값으로 처리
date_cols = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'begin_month']
for col in date_cols:
    train[col] = np.abs(train[col])

### 5. 나이 처리하기

In [ ]:
train['Age'] = (train['DAYS_BIRTH'] // 365)

In [ ]:
train['Age_G'] = (train['Age'] // 10)

0        38
1        31
2        52
3        41
4        41
         ..
26393    33
26394    41
26395    27
26396    27
26397    53
Name: Age, Length: 26398, dtype: int64

### 6. 아웃라이어 처리하기

#### 6-1 family_size

In [ ]:
train = train[(train['family_size'] < 6)]
train = train.reset_index(drop=True) # 필요할까